In [1]:
import sqlite3
import tkinter
from tkinter import ttk
from tkinter import messagebox
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import joblib
from tkinter import *
from sklearn.preprocessing import StandardScaler

#database ########
def reverse(tuples):
    new_tup = tuples[::-1]
    return new_tup


def insert( id, name, price, quantity):
    conn = sqlite3.connect("data.db")
    cursor = conn.cursor()

    cursor.execute("""CREATE TABLE IF NOT EXISTS 
    inventory(itemId TEXT, itemName TEXT, itemPrice TEXT, itemQuantity TEXT)""")

    cursor.execute("INSERT INTO inventory VALUES ('" + str(id) + "','" + str(name) + "','" + str(price) + "','" + str(quantity) + "')")
    conn.commit()


def delete(data):
    conn = sqlite3.connect("data.db")
    cursor = conn.cursor()

    cursor.execute("""CREATE TABLE IF NOT EXISTS 
        inventory(itemId TEXT, itemName TEXT, itemPrice TEXT, itemQuantity TEXT)""")

    cursor.execute("DELETE FROM inventory WHERE itemId = '" + str(data) + "'")
    conn.commit()


def update(id, name, price, quantity,  idName):
    conn = sqlite3.connect("data.db")
    cursor = conn.cursor()

    cursor.execute("""CREATE TABLE IF NOT EXISTS 
        inventory(itemId TEXT, itemName TEXT, itemPrice TEXT, itemQuantity TEXT)""")

    cursor.execute("UPDATE inventory SET itemId = '" + str(id) + "', itemName = '" + str(name) + "', itemPrice = '" + str(price) + "', itemQuantity = '" + str(quantity) + "' WHERE itemId='"+str(idName)+"'")
    conn.commit()


def read():
    conn = sqlite3.connect("data.db")
    cursor = conn.cursor()

    cursor.execute("""CREATE TABLE IF NOT EXISTS 
        inventory(itemId TEXT, itemName TEXT, itemPrice TEXT, itemQuantity TEXT)""")

    cursor.execute("SELECT * FROM inventory")
    results = cursor.fetchall()
    conn.commit()
    return results


def insert_data():
    itemId = str(entryId.get())
    itemName = str(entryName.get())
    itemPrice = str(entryPrice.get())
    itemQuantity = str(entryQuantity.get())
    if itemId == "" or itemName == " ":
        print("Error Inserting Id")
    if itemName == "" or itemName == " ":
        print("Error Inserting Name")
    if itemPrice == "" or itemPrice == " ":
        print("Error Inserting Price")
    if itemQuantity == "" or itemQuantity == " ":
        print("Error Inserting Quantity")
    else:
        insert(str(itemId), str(itemName), str(itemPrice), str(itemQuantity))

    for data in my_tree.get_children():
        my_tree.delete(data)

    for result in reverse(read()):
        my_tree.insert(parent='', index='end', iid=result, text="", values=(result), tag="orow")

    my_tree.tag_configure('orow', background='#EEEEEE')
    my_tree.grid(row=1, column=5, columnspan=4, rowspan=5, padx=10, pady=10)


def delete_data():
    selected_item = my_tree.selection()[0]
    deleteData = str(my_tree.item(selected_item)['values'][0])
    delete(deleteData)

    for data in my_tree.get_children():
        my_tree.delete(data)

    for result in reverse(read()):
        my_tree.insert(parent='', index='end', iid=result, text="", values=(result), tag="orow")

    my_tree.tag_configure('orow', background='#EEEEEE')
    my_tree.grid(row=1, column=5, columnspan=4, rowspan=5, padx=10, pady=10)

def update_data():
    selected_item = my_tree.selection()[0]
    update_name = my_tree.item(selected_item)['values'][0]
    update(entryId.get(), entryName.get(), entryPrice.get(), entryQuantity.get(), update_name)

    for data in my_tree.get_children():
        my_tree.delete(data)

    for result in reverse(read()):
        my_tree.insert(parent='', index='end', iid=result, text="", values=(result), tag="orow")

    my_tree.tag_configure('orow', background='#EEEEEE')
    my_tree.grid(row=1, column=5, columnspan=4, rowspan=5, padx=10, pady=10)

#database ########

def predict_oil_yield():
    #input
    hdpe = float(hdpe_entry.get())
    ldpe = float(ldpe_entry.get())
    pp = float(pp_entry.get())
    ps = float(ps_entry.get())
    pvc = float(pvc_entry.get())
    pet = float(pet_entry.get())
    particleSize = float(ParticleSize_entry.get())
    feedSize = float(FeedSize_entry.get())
    catalyst = Catalyst_combobox.get()
    if(catalyst == "Yes"):
        catalyst = 1
    else:
        catalyst = 0
    float(catalyst)
    
    #finding optimal temperature and heating rate
    model = joblib.load('ann_model.joblib')
    x_scaler = joblib.load('x_scaler.joblib')
    y_scaler = joblib.load('y_scaler.joblib')

    feature_names = ["HDPE", "LDPE", "PP", "PS", "PVC", "PET", "Temperature",
                    "Heating_Rate", "Particle_Size", "Feed_Size", "Catalyst"]


    grid1 = [400,450,500,550]
    grid2 = [5,10,15,20]

    optimal_input = None
    max_output = float('-inf')

    for Temperature in grid1:
        for Heating_Rate in grid2:
            
            input_array = [[hdpe,ldpe,pp,ps,pvc,pet,Temperature,Heating_Rate,particleSize,feedSize,catalyst]]
            x = x_scaler.transform(input_array)
            
            output_value = model.predict(x)
            output_value = y_scaler.inverse_transform(output_value)

            if output_value > max_output:
                max_output = output_value
                optimal_input = [Temperature,Heating_Rate]
                
    formatted_values = [f"{value:.1f}" for value in max_output.flatten()]
    
    output_label.config(text="The optimal Temperature and Heating rate is:  \n\n"+
                        "Temperature: {:.2f}".format(optimal_input[0])+
                        "   Heating Rate: {:.2f}".format(optimal_input[1])+
                       "\nWith Predicted Maximum Oil Yield of: " + str(formatted_values[0]))
    
    
########################################################
window = tkinter.Tk()
window.title("SITCSP")

my_tree = ttk.Treeview(window)

frame = tkinter.Frame(window)
frame.pack()

#Plastic Type
user_info_frame =tkinter.LabelFrame(frame, text="Insert Plastic Type")
user_info_frame.grid(row= 0, column=0, padx=5, pady=10)

hdpe_label = tkinter.Label(user_info_frame, text="HDPE(wt%)")
hdpe_label.grid(row=0, column=0)
ldpe_label = tkinter.Label(user_info_frame, text="LDPE(wt%)")
ldpe_label.grid(row=0, column=1)
pp_label = tkinter.Label(user_info_frame, text="PP(wt%)")
pp_label.grid(row=0, column=2)
ps_label = tkinter.Label(user_info_frame, text="PS(wt%)")
ps_label.grid(row=2, column=0)
pvc_label = tkinter.Label(user_info_frame, text="PVC(wt%)")
pvc_label.grid(row=2, column=1)
pet_label = tkinter.Label(user_info_frame, text="PET(wt%)")
pet_label.grid(row=2, column=2)

hdpe_entry = tkinter.Entry(user_info_frame)
ldpe_entry = tkinter.Entry(user_info_frame)
pp_entry = tkinter.Entry(user_info_frame)
ps_entry = tkinter.Entry(user_info_frame)
pvc_entry = tkinter.Entry(user_info_frame)
pet_entry = tkinter.Entry(user_info_frame)

hdpe_entry.grid(row=1, column=0)
ldpe_entry.grid(row=1, column=1)
pp_entry.grid(row=1, column=2)
ps_entry.grid(row=3, column=0)
pvc_entry.grid(row=3, column=1)
pet_entry.grid(row=3, column=2)

for widget in user_info_frame.winfo_children():
    widget.grid_configure(padx=63, pady=8)

# Additional Info #
courses_frame = tkinter.LabelFrame(frame, text = "Additional Info")
courses_frame.grid(row=1, column=0, sticky="news", padx=20, pady=10)

ParticleSize_label = tkinter.Label(courses_frame, text="Particle Size(mm)")
ParticleSize_label.grid(row=0, column=1)
FeedSize_label = tkinter.Label(courses_frame, text="Feed Size(g)")
FeedSize_label.grid(row=0, column=2)

ParticleSize_entry = tkinter.Entry(courses_frame)
FeedSize_entry = tkinter.Entry(courses_frame)
ParticleSize_entry.grid(row=1, column=1)
FeedSize_entry.grid(row=1, column=2)

Catalyst_label = tkinter.Label(courses_frame, text="#Catalyst")
Catalyst_combobox = ttk.Combobox(courses_frame, values=["Yes","No"])
Catalyst_label.grid(row=0, column=3)
Catalyst_combobox.grid(row=1, column=3)

for widget in courses_frame.winfo_children():
    widget.grid_configure(padx=15, pady=8)

# Output
terms_frame = tkinter.LabelFrame(frame, text="Output")
terms_frame.grid(row=2, column=0, sticky="news", padx=20, pady=20)

output_label = ttk.Label(terms_frame, text="The optimal Temperature and Heating rate is: ")
output_label.grid(row=0,column=0)

buttonPredict = tkinter.Button(terms_frame, text="Predict", command= predict_oil_yield)
buttonPredict.grid(row=1, column=0, sticky="news", padx=20, pady=0)

for widget in terms_frame.winfo_children():
    widget.grid_configure(padx=15, pady=20)

# Database
databaseframe = tkinter.LabelFrame(frame, text = "Database")
databaseframe.grid(row = 3, column = 0, sticky="news", padx=10, pady=10)

idLabel = Label(databaseframe, text="Temperature")
nameLabel = Label(databaseframe, text="Heating Rate")
priceLabel = Label(databaseframe, text="Value Predicted")
quantityLabel = Label(databaseframe, text="Value Acquired")
idLabel.grid(row=1, column=0, padx=0, pady=5)
nameLabel.grid(row=2, column=0, padx=0, pady=5)
priceLabel.grid(row=3, column=0, padx=0, pady=5)
quantityLabel.grid(row=4, column=0, padx=0, pady=5)

entryId = Entry(databaseframe, width=10, bd=1)
entryName = Entry(databaseframe, width=10, bd=1)
entryPrice = Entry(databaseframe, width=10, bd=1)
entryQuantity = Entry(databaseframe, width=10, bd=1)
entryId.grid(row=1, column=1, columnspan=3, padx=0, pady=5)
entryName.grid(row=2, column=1, columnspan=3, padx=0, pady=5)
entryPrice.grid(row=3, column=1, columnspan=3, padx=0, pady=5)
entryQuantity.grid(row=4, column=1, columnspan=3, padx=0, pady=5)

buttonEnter = Button(
    databaseframe, text="Enter", padx=5, pady=5, width=5,
    bd=1, command=insert_data)
buttonEnter.grid(row=5, column=0, columnspan=1)

buttonUpdate = Button(
    databaseframe, text="Update", padx=5, pady=5, width=5,
    bd=1, command=update_data)
buttonUpdate.grid(row=5, column=1, columnspan=1)

buttonDelete = Button(
    databaseframe, text="Delete", padx=5, pady=5, width=5,
    bd=1, command=delete_data)
buttonDelete.grid(row=5, column=2, columnspan=1)

my_tree = ttk.Treeview(databaseframe)
style = ttk.Style()
style.configure("Treeview.Heading")

my_tree['columns'] = ("Temperature", "Heating Rate", 
                      "Predicted Oil Yield", "Acquired Oil Yield")
my_tree.column("#0", width=0, stretch=NO)
my_tree.column("Temperature", anchor=W, width=100)
my_tree.column("Heating Rate", anchor=W, width=100)
my_tree.column("Predicted Oil Yield", anchor=W, width=150)
my_tree.column("Acquired Oil Yield", anchor=W, width=150)
my_tree.heading("Temperature", text="Temperature", anchor=W)
my_tree.heading("Heating Rate", text="Heating Rate", anchor=W)
my_tree.heading("Predicted Oil Yield", text="Predicted Oil Yield", anchor=W)
my_tree.heading("Acquired Oil Yield", text="Acquired Oil Yield", anchor=W)

for data in my_tree.get_children():
    my_tree.delete(data)

for result in reverse(read()):
    my_tree.insert(parent='', index='end', iid=0, text="", values=(result), tag="orow")

my_tree.tag_configure('orow', background='#EEEEEE')
my_tree.grid(row=1, column=5, columnspan=4, rowspan=5, padx=20, pady=10)

for widget in databaseframe.winfo_children():
    widget.grid_configure(padx=10, pady=10)
 
window.mainloop()

1/1 [==============================] - 0s 37ms/step
